# Perform Doc2Vec

# Read data

This notebook trains a doc2vec model on the entire notebook dataset. 

# Import modules

In [25]:
import pandas as pd
import numpy as np
import os
import gensim
from gensim.models.doc2vec import TaggedDocument

# Read data

In [26]:
df = pd.read_csv("../data/smalldf-1000notebooks.csv")
df['filename'] = df['filename'].astype(str)

In [27]:
df = df.replace(np.nan, '', regex=True)



In [28]:
df.head(10)
len(df)

30368

### Ignore markdown, only consider code for now

In [29]:
subdf = df
subdf = df[df.cell_type == "code"] 

### Calculate the line number for each notebook

In [30]:
subdf['cell_num']=subdf.groupby(['filename']).cumcount()+1
subdf['filename_with_cellnum'] = subdf['filename'] + "_" + subdf['cell_num'].astype(str)
subdf['filename_with_cellnum'] = subdf['filename_with_cellnum'].astype(str)

<ipython-input-30-21e3d0f3a8e1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subdf['cell_num']=subdf.groupby(['filename']).cumcount()+1
<ipython-input-30-21e3d0f3a8e1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subdf['filename_with_cellnum'] = subdf['filename'] + "_" + subdf['cell_num'].astype(str)
<ipython-input-30-21e3d0f3a8e1>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in

In [31]:
df.source.isnull().values.any()

False

# Preprocessing
## Generate input data from raw source

In [32]:
from gensim.utils import simple_preprocess

class MyDataframeCorpus(object):
    def __init__(self, source_df, text_col, tag_col):
        self.source_df = source_df
        self.text_col = text_col
        self.tag_col = tag_col

    def __iter__(self):
        for i, row in self.source_df.iterrows():
            yield TaggedDocument(words=simple_preprocess(row[self.text_col]), 
                                 tags=[row[self.tag_col]])


## Train the model

In [33]:
train_corpus = MyDataframeCorpus(subdf, 'source', 'filename_with_cellnum')

In [34]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=768,  epochs=40)

In [35]:
model.build_vocab(train_corpus)

In [36]:
model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

## Test inference

In [37]:
testinput = "import numpy as np"
vector = model.infer_vector(testinput.split(" "))
# print(vector)

In [38]:
sims = model.docvecs.most_similar([vector])
print(sims)

for result in sims[0:3]:
    display(df[df.filename_with_cellnum == result[0]]['source'])

[('20507743_35', 0.8999701738357544), ('30685091_14', 0.8642010688781738), ('358139_1', 0.8553161025047302), ('43070884_15', 0.8261731863021851), ('43070884_12', 0.8153966665267944), ('14387505_1', 0.8075113892555237), ('4222411_1', 0.8048861026763916), ('19576968_3', 0.7958853840827942), ('30655012_2', 0.7916973829269409), ('882622_9', 0.7886699438095093)]


15240    def auroc(y_true, y_pred):\n     return tf.py_...
Name: source, dtype: object

17053    from numpy.random import seed\n from numpy.ran...
Name: source, dtype: object

5415    Sparse matrix with explicit zeroes!\n --------...
Name: source, dtype: object

# Save the model

In [39]:
model.save("../model/notebook-doc2vec-model-apr7-1000notebooks.model")

In [17]:
from gensim.models.doc2vec import Doc2Vec


In [19]:
model1 = Doc2Vec.load("../model/notebook-doc2vec-model-apr7-small.model")